# Choosing parameters

This notebook documents the choice of simulation parameters including:

* Length of warm-up period
* Number of replications
* Number of CPU cores
* Sampling distributions

The run time is provided at the end of the notebook.

Credit:

* Code for choice of warm-up period, replication number and distributions was adapted from Tom Monks (2024) [HPDM097 - Making a difference with health data](https://github.com/health-data-science-OR/stochastic_systems) (MIT License).
* Code for running the model with a varying number of CPU cores was adapted from Sammi Rosser and Dan Chalk (2024) [HSMA - the
    little book of DES](https://github.com/hsma-programme/hsma6_des_book) (MIT License).

License:

* This project is licensed under the MIT License. See the `LICENSE` file for more details.

## Set-up

Load notebook linters.

In [1]:
%load_ext pycodestyle_magic

In [2]:
%pycodestyle_on

Load required packages.

In [3]:
# To ensure any updates to `simulation/` are fetched without needing to restart
# the notebook environment, reload `simulation/` before execution of each cell
%load_ext autoreload
%autoreload 1
%aimport simulation

In [4]:
from simulation.model import Defaults, Trial, summary_stats
import time
from IPython.display import display
import numpy as np
import plotly.express as px
import polars as pl
import warnings

Start timer.

In [5]:
notebook_start_time = time.time()

## Choosing length of warm-up period

A suitable length for the warm-up period can be determined using the **time series inspection approach**. This involves looking at performance measures over time to identify when the system is exhibiting **steady state behaviour** (even though the system will never truly reach a "steady state").

If we simply plot the mean result at regular intervals, this would vary too much. Therefore, we plot the **cumulative mean** of the performance measure, and look for the point at which this **smoothes out and stabilises**. This indicates the point for the warm-up period to end.

This should be assessed when running the model using the base case parameters. If these change, you should reassess the appropriate warm-up period.

In [6]:
def cum_mean(x):
    """
    Calculate cumulative/rolling mean for a polars Series.

    Arguments:
        x (pl.Series):
            Series to apply calculation to

    Returns:
        pl.Series with cumulative mean.
    """
    return pl.col(x).cum_sum().truediv(pl.col(x).cum_count())

In [7]:
def time_series_inspection(data_collection_period, warm_up=None):
    """
    Time series inspection method for determining length of warm-up.

    Arguments:
        data_collection_period (float):
            Length of time to run the simulation for.
        warm_up (float, optional):
            Location on X axis to plot vertical red line indicating the chosen
            warm-up period. Defaults to None, which will not plot a line.
    """
    # Use default parameters, but with no warm-up and specified run length,
    # and with no replications
    param = Defaults()
    param.warm_up_period = 0
    param.data_collection_period = data_collection_period
    param.number_of_runs = 1
    # display(param.__dict__)

    # Run model
    choose_warmup = Trial(param)
    choose_warmup.run_trial()

    # Filter to nurse results
    nurse = choose_warmup.interval_audit_df.filter(
        pl.col('resource_name') == 'nurse')

    # Define columns to analyse
    plot = {
        'utilisation': 'Cumulative mean nurse utilisation',
        'running_mean_wait_time': 'Running mean nurse wait time'
    }
    for var, label in plot.items():
        # Filter to relevant columns
        df = nurse.select(['simulation_time', var])

        # For utilisation, calculate cumulative mean (not necessary
        # for wait time as that is already a running mean)
        if var == 'utilisation':
            cumulative = df.with_columns(cum_mean(var))
        elif var == 'running_mean_wait_time':
            cumulative = df

        # Create plot. If specified, add vertical line to indicate suggested
        # warm-up length
        fig = px.line(cumulative, x='simulation_time', y=var)
        fig.update_layout(
            xaxis_title='Run time (minutes)',
            yaxis_title=label,
            showlegend=False
        )
        if warm_up is not None:
            fig.add_vline(x=warm_up, line_color='red', line_dash='dash')
        fig.show()

Having run the model for three days, it appears to reach a steady state at around 2500 minutes.

In [8]:
time_series_inspection(data_collection_period=1440*3, warm_up=2520)

However, it is important to look far ahead - so we run it for more days, and find actually a later warm-up is more appropriate.

In [9]:
time_series_inspection(data_collection_period=1440*40, warm_up=1440*13)

## Choosing the number of replications

The **confidence interval method** can be used to select the number of replications to run. The more replications you run, the narrower your confidence interval becomes, leading to a more precise estimate of the model's mean performance.

First, you select a desired confidence interval - for example, 95%. Then, run the model with an increasing number of replications, and identify the number required to achieve that precision in the estimate of a given metric - and also, to maintain that precision (as the intervals may converge or expand again later on).

This method is less useful for values very close to zero - so, for example, when using utilisation (which ranges from 0 to 1) it is recommended to multiple values by 100.

When selecting the number of replications you should repeat the analysis for all performance measures and select the highest value as your number of replications.

In [10]:
def confidence_interval_method(replications, metric, desired_precision,
                               min_rep=None):
    """
    Use the confidence interval method to select the number of replications.

    Arguments:
        replications (int):
            Number of times to run the model.
        metric (string):
            Name of performance metric to assess.
        desired_precision (float):
            Desired mean deviation from confidence interval.
        min_rep (int):
            A suggested minimum number of replications.
    """
    param = Defaults()
    param.number_of_runs = replications
    choose_rep = Trial(param)
    choose_rep.run_trial()

    # If mean of metric is less than 1, multiply by 100
    if choose_rep.trial_results_df.select(pl.mean(metric)).item() < 1:
        choose_rep.trial_results_df = choose_rep.trial_results_df.with_columns(
            (pl.col(metric)*100).alias(f'adj_{metric}')
        )

    # Initialise list to store the results
    cumulative_list = []

    # For each row in the dataframe, filter to rows up to the i-th replication
    # then perform calculations
    for i in range(1, replications+1):
        mean, std_dev, ci_lower, ci_upper = summary_stats(
            choose_rep
            .trial_results_df
            .with_row_index()
            .filter(pl.col('index') < i)
            .select(metric))
        deviation = ((
            ci_upper.item()-mean.item())/mean.item())*100 if i > 1 else np.nan
        cumulative_list.append({
            'replications': i,
            'cumulative_mean': mean.item(),
            'cumulative_std': std_dev.item() if i > 1 else np.nan,
            'lower_ci': ci_lower.item() if i > 1 else np.nan,
            'upper_ci': ci_upper.item() if i > 1 else np.nan,
            'perc_deviation': deviation
        })
    cumulative = pl.DataFrame(cumulative_list)
    display(cumulative)

    # Get minimum number of replications where deviation is less than target
    try:
        n_reps = (cumulative
                  .filter(pl.col('perc_deviation') <= desired_precision*100)
                  .head(1)
                  .select('replications')
                  .item())
        print(f'Reached desired precision ({desired_precision}) in {n_reps} ' +
              'replications.')
    except IndexError:
        warnings.warn(f'Running {replications} replications did not reach' +
                      f'desired precision ({desired_precision}).')

    # Plot the cumulative mean and confidence interval
    fig = px.line(cumulative,
                  x='replications',
                  y=['cumulative_mean', 'lower_ci', 'upper_ci'])
    fig.update_layout(
        xaxis_title='Number of replications',
        yaxis_title=metric
    )
    if min_rep is not None:
        fig.add_vline(x=min_rep, line_color='red', line_dash='dash')
    fig.show()

In [11]:
confidence_interval_method(
    replications=20,
    metric='mean_time_with_nurse',
    desired_precision=0.05,
    min_rep=3
)

replications,cumulative_mean,cumulative_std,lower_ci,upper_ci,perc_deviation
i64,f64,f64,f64,f64,f64
1,9.842268,NaN,NaN,NaN,NaN
2,9.951374,0.1543,8.565044,11.337705,13.931046
3,9.942591,0.110162,9.668933,10.216249,2.752379
4,9.941208,0.08999,9.798014,10.084401,1.440402
5,9.956147,0.084791,9.850865,10.061429,1.057456
…,…,…,…,…,…
16,9.997145,0.114738,9.936005,10.058284,0.611569
17,9.986125,0.120027,9.924413,10.047837,0.617979
18,9.986293,0.116445,9.928386,10.0442,0.579864


Reached desired precision (0.05) in 3 replications.


It is important to check ahead 10-20 replications, to check that the 5% precision is maintained. For this example, 3 replications is < 5, but then it quickly goes back up, and is not stable until 31 replications.

In [12]:
confidence_interval_method(
    replications=50,
    metric='mean_q_time_nurse',
    desired_precision=0.05,
    min_rep=31
)

replications,cumulative_mean,cumulative_std,lower_ci,upper_ci,perc_deviation
i64,f64,f64,f64,f64,f64
1,0.504541,NaN,NaN,NaN,NaN
2,0.509346,0.006795,0.448295,0.570396,11.986073
3,0.513976,0.009348,0.490753,0.537198,4.518132
4,0.505269,0.019013,0.475015,0.535522,5.987597
5,0.496507,0.025593,0.464729,0.528284,6.400293
…,…,…,…,…,…
46,0.521207,0.076469,0.498499,0.543916,4.356909
47,0.519995,0.076089,0.497654,0.542335,4.296302
48,0.520911,0.075542,0.498976,0.542846,4.210927


Reached desired precision (0.05) in 3 replications.


In [13]:
confidence_interval_method(
    replications=20,
    metric='mean_nurse_utilisation',
    desired_precision=0.05,
    min_rep=3
)

replications,cumulative_mean,cumulative_std,lower_ci,upper_ci,perc_deviation
i64,f64,f64,f64,f64,f64
1,0.499639,NaN,NaN,NaN,NaN
2,0.500815,0.001663,0.485873,0.515756,2.983507
3,0.49992,0.001946,0.495086,0.504753,0.966831
4,0.499495,0.001802,0.496628,0.502362,0.573914
5,0.49897,0.001952,0.496546,0.501394,0.485842
…,…,…,…,…,…
16,0.500591,0.006697,0.497022,0.50416,0.712919
17,0.499774,0.007308,0.496016,0.503531,0.751859
18,0.499531,0.007164,0.495968,0.503094,0.71322


Reached desired precision (0.05) in 2 replications.


## Run time with varying number of CPU cores

In [14]:
# Run with 1 to 8 cores
speed = []
for i in range(1, 9):
    start_time = time.time()

    param = Defaults()
    param.cores = i
    my_trial = Trial(param)
    my_trial.run_trial()

    run_time = round((time.time() - start_time), 3)
    speed.append({'cores': i, 'run_time': run_time})

# Display and plot time by number of cores
timing_results = pl.DataFrame(speed)
print(timing_results)
fig = px.line(timing_results, x='cores', y='run_time')
fig.update_layout(
    xaxis_title='Number of cores',
    yaxis_title='Run time (seconds)'
)
fig.show()

shape: (8, 2)
┌───────┬──────────┐
│ cores ┆ run_time │
│ ---   ┆ ---      │
│ i64   ┆ f64      │
╞═══════╪══════════╡
│ 1     ┆ 4.237    │
│ 2     ┆ 2.939    │
│ 3     ┆ 2.114    │
│ 4     ┆ 1.761    │
│ 5     ┆ 1.662    │
│ 6     ┆ 1.648    │
│ 7     ┆ 1.495    │
│ 8     ┆ 0.935    │
└───────┴──────────┘


## Run time

In [15]:
# Get run time in seconds
notebook_end_time = time.time()
runtime = round(notebook_end_time - notebook_start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 21s
